<a href="https://colab.research.google.com/github/ash-netizen/Mental_Fatigue/blob/main/ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
!pip install -q ktrain

     |████████████████████████████████| 6.8MB 16.2MB/s 
     |████████████████████████████████| 983kB 42.4MB/s 
     |████████████████████████████████| 266kB 46.8MB/s 
     |████████████████████████████████| 1.9MB 38.4MB/s 
     |████████████████████████████████| 1.2MB 41.4MB/s 
     |████████████████████████████████| 471kB 36.8MB/s 
     |████████████████████████████████| 3.3MB 42.4MB/s 
     |████████████████████████████████| 901kB 28.0MB/s 


In [3]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [4]:
import ktrain
from ktrain import tabular
import pandas as pd
import numpy as np
ktrain.__version__

'0.26.2'

In [133]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [134]:
train = train.dropna()
train.head(1)

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Age,Tenure,Vacations taken,Designation,Average Hours worked per day,Employee satisfaction score,Mental Fatigue Score
0,fffe32003000360033003200,2008-09-30,Female,Service,No,52,18,8,2,3.0,3.8,0.16


In [135]:
data_columns = ['WFH Setup Available',
                'Designation'
                , 'Average Hours worked per day',
                'Employee satisfaction score'
                , 'Mental Fatigue Score'
               ]

train_df = train[data_columns]

In [136]:
trn, val, preproc = tabular.tabular_from_df(train_df, label_columns=['Mental Fatigue Score'], random_state=42, is_regression=True )


processing train: 16836 rows x 5 columns
['Mental Fatigue Score']
   Mental Fatigue Score
0                  0.16
3                  0.20
4                  0.52
5                  0.29
6                  0.62

The following integer column(s) are being treated as categorical variables:
['Designation']
To treat any of these column(s) as numerical, cast the column to float in DataFrame or CSV
 and re-run tabular_from* function.

processing test: 1834 rows x 5 columns
['Mental Fatigue Score']
    Mental Fatigue Score
1                   0.36
16                  0.39
44                  0.59
46                  0.94
57                  0.61


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:580: UserWarning: Task is being treated as REGRESSION because either class_names argument was not supplied or is_regression=True. If this is incorrect, change accordingly.
  'either class_names argument was not supplied or is_regression=True. ' + \


In [137]:
tabular.print_tabular_regression_models()


mlp: a configurable multilayer perceptron with categorical variable embeddings [https://arxiv.org/abs/1604.06737]


In [138]:
model = tabular.tabular_regression_model('mlp', trn, metrics=[tf.keras.metrics.RootMeanSquaredError()])
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=256)

done.


In [139]:
learner.fit_onecycle(4e-3, 35)




begin training using onecycle policy with max lr of 0.004...
Epoch 1/35
66/66 [==============================] - 3s 39ms/step - loss: 0.0742 - root_mean_squared_error: 0.2625 - val_loss: 0.0052 - val_root_mean_squared_error: 0.0725
Epoch 2/35
66/66 [==============================] - 2s 36ms/step - loss: 0.0043 - root_mean_squared_error: 0.0655 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0633
Epoch 3/35
66/66 [==============================] - 2s 36ms/step - loss: 0.0038 - root_mean_squared_error: 0.0617 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0628
Epoch 4/35
66/66 [==============================] - 2s 36ms/step - loss: 0.0043 - root_mean_squared_error: 0.0655 - val_loss: 0.0041 - val_root_mean_squared_error: 0.0642
Epoch 5/35
66/66 [==============================] - 2s 36ms/step - loss: 0.0039 - root_mean_squared_error: 0.0628 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0635
Epoch 6/35
66/66 [==============================] - 2s 37ms/step - loss: 0.0040 - 

In [140]:
test_df = test[data_columns[:-1]]

In [141]:
predictor = ktrain.get_predictor(learner.model, preproc)
preds = predictor.predict(test_df)

['Mental Fatigue Score']
   Mental Fatigue Score
0                   0.0
1                   0.0
2                   0.0
3                   0.0
4                   0.0


In [142]:
sub = pd.read_csv("test.csv")
sub = sub.loc[:, ["Employee ID"]]

In [143]:
sub['Mental Fatigue Score'] = preds
with open('KT_submission.csv', 'w') as f:
    sub.to_csv(f, index=False, header=True)

from google.colab import files
files.download('KT_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>